In [1]:
import pandas as pd
from itertools import combinations, permutations
from numpy import cos, sin, arcsin, sqrt
from math import radians

In [2]:
stations_2017 = pd.read_csv('data/2017/Stations_2017.csv')
stations_2018 = pd.read_csv('data/2018/Stations_2018.csv')
stations_2019 = pd.read_csv('data/2019/Stations_2019.csv')

In [3]:
for station in [stations_2017, stations_2018, stations_2019]:
    print(len(station))

546
552
619


In [4]:
stations_2019 = stations_2019.head(10)

In [5]:
stations_2019

,Code,name,latitude,longitude
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478228,-73.569651
1,4000,Jeanne-d'Arc / Ontario,45.549598,-73.541874
2,4001,Graham / Brookfield,45.520075,-73.629776
3,4002,Graham / Wicksteed,45.516937,-73.640483
4,5002,St-Charles / Montarville,45.533682,-73.515261
5,5003,Place Longueuil,45.528410,-73.517166
6,5004,St-Charles / Grant,45.539824,-73.508752
7,5005,St-Charles / St-Jean,45.536907,-73.511914
8,5006,Collège Édouard-Montpetit (de Gentilly / de No...,45.537226,-73.495067
9,5007,Métro Longueuil - Université de Sherbrooke,45.523319,-73.520127


In [6]:
station_0 = stations_2019.iloc[:1]

In [7]:
station_0

,Code,name,latitude,longitude
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478228,-73.569651


In [8]:
stations_2019

,Code,name,latitude,longitude
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478228,-73.569651
1,4000,Jeanne-d'Arc / Ontario,45.549598,-73.541874
2,4001,Graham / Brookfield,45.520075,-73.629776
3,4002,Graham / Wicksteed,45.516937,-73.640483
4,5002,St-Charles / Montarville,45.533682,-73.515261
5,5003,Place Longueuil,45.528410,-73.517166
6,5004,St-Charles / Grant,45.539824,-73.508752
7,5005,St-Charles / St-Jean,45.536907,-73.511914
8,5006,Collège Édouard-Montpetit (de Gentilly / de No...,45.537226,-73.495067
9,5007,Métro Longueuil - Université de Sherbrooke,45.523319,-73.520127


In [9]:
density = 0
for row in stations_2019.iterrows():
    lon1 = station_0['longitude']
    lat1 = station_0['latitude']

    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, row[1]['longitude'], row[1]['latitude']])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c

    if (km < 10):
        print(km)
        density += 1

print(density)

0.0
8.220636419699014
6.599693100133113
6.9959081672659
7.47782022254739
6.914316363577335
8.32733812484655
7.920780240549544
8.759091565249426
6.323472873337828
10


In [10]:
def calculate_station_density(df, threshhold):
    '''
    This function is used to add a station density (number of stations in a specified radius of another station) for each station.
    
    :param DataFrame df: The original stations dataframe
    :param int threshhold: Threshold for specified radius in km
    :return pd.DataFrame: dataframe with densities for each station
    '''
    
    density_frame = pd.DataFrame(columns=['code', 'density'])
    
    for base_station in df.iterrows():
        density = 0 # init density for base_station

        # get geo coords for base station
        lon1 = base_station[1]['longitude']
        lat1 = base_station[1]['latitude']

        print("Base Station: {lon1} {lat1}".format(lon1=lon1, lat1=lat1))

        df_excluding_base_station = df.drop(base_station[0])

        for station_x in df_excluding_base_station.iterrows():

            # calculate distance
            lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, station_x[1]['longitude'], station_x[1]['latitude']])
            dlon = lon2 - lon1 
            dlat = lat2 - lat1 
            a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
            c = 2 * arcsin(sqrt(a)) 
            km = 6367 * c
            print("Code: {code} / lon2: {lon2} / lat2: {lat2} KM: {km}".format(code=station_x[1]['Code'], km=km, lon2=station_x[1]['longitude'], lat2=station_x[1]['latitude']))

            if km < threshhold:
                density += 1

        density_frame = density_frame.append({
            'code': base_station[1]['Code'],
            'density': density
        }, ignore_index=True)
    
    return density_frame


In [11]:
calculate_station_density(stations_2019, 10)

Base Station: -73.56965124607086 45.47822787309145
Code: 4000 / lon2: -73.54187428951263 / lat2: 45.54959767887176 KM: 8.220636419699014
Code: 4001 / lon2: -73.6297756433487 / lat2: 45.52007477080967 KM: 8573.596913187159
Code: 4002 / lon2: -73.64048302173615 / lat2: 45.51693654712809 KM: 8733.528925066528
Code: 5002 / lon2: -73.51526074111462 / lat2: 45.53368199320325 KM: 8727.150571079539
Code: 5003 / lon2: -73.51716578006744 / lat2: 45.52841043752244 KM: 8727.223131284798
Code: 5004 / lon2: -73.508752 / lat2: 45.539824 KM: 8726.845292966236
Code: 5005 / lon2: -73.51191401481627 / lat2: 45.53690747306937 KM: 8727.019114554221
Code: 5006 / lon2: -73.49506705999374 / lat2: 45.53722603812578 KM: 8725.743457961762
Code: 5007 / lon2: -73.52012693881989 / lat2: 45.52331869398737 KM: 8727.332678170153
Base Station: -73.54187428951263 45.54959767887176
Code: 10002 / lon2: -73.56965124607086 / lat2: 45.47822787309145 KM: 8.220636419699014
Code: 4001 / lon2: -73.6297756433487 / lat2: 45.520074

C:\Users\NicoDöring\AppData\Local\Temp\ipykernel_30684\4060841319.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  density_frame = density_frame.append({
C:\Users\NicoDöring\AppData\Local\Temp\ipykernel_30684\4060841319.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  density_frame = density_frame.append({
C:\Users\NicoDöring\AppData\Local\Temp\ipykernel_30684\4060841319.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  density_frame = density_frame.append({
C:\Users\NicoDöring\AppData\Local\Temp\ipykernel_30684\4060841319.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  density_frame = density_frame.append({
C:\Users\Nic

,code,density
0,10002,1
1,4000,1
2,4001,1
3,4002,1
4,5002,1
5,5003,1
6,5004,1
7,5005,1
8,5006,1
9,5007,1


In [22]:
distance_frame = pd.DataFrame(columns=['base_station_code', 'target_station_code', 'distance'])
density_frame = pd.DataFrame(columns=['station_code', 'density'])


for index in permutations(stations_2019.index, 2):

    base_station_code = stations_2019.at[index[0], 'Code']
    lon1 = stations_2019.iloc[[index[0]]]['longitude']
    lat1 = stations_2019.iloc[[index[0]]]['latitude']

    target_station_code = stations_2019.at[index[1], 'Code']
    lon2 = stations_2019.iloc[[index[1]]]['longitude']
    lat2 = stations_2019.iloc[[index[1]]]['latitude']

    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c

    distance_frame = distance_frame.append({
        'base_station_code': base_station_code,
        'target_station_code':target_station_code,
        'distance': km
    }, ignore_index=True)

    distance_frame[['base_station_code', 'target_station_code']] = distance_frame[['base_station_code', 'target_station_code']].astype('int')

unique_base_station_codes = list(distance_frame.base_station_code.unique())

for base_station_code in unique_base_station_codes:

    dict = {
        'station_code': base_station_code,
        'density': len(distance_frame[(distance_frame['base_station_code']==base_station_code) & (distance_frame['distance'] < 7)])
    }

    df2 = pd.DataFrame([dict])

    density_frame = pd.concat([density_frame, df2])

density_frame

C:\Users\NicoDöring\AppData\Local\Temp\ipykernel_30684\2029092873.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  distance_frame = distance_frame.append({
C:\Users\NicoDöring\AppData\Local\Temp\ipykernel_30684\2029092873.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  distance_frame = distance_frame.append({
C:\Users\NicoDöring\AppData\Local\Temp\ipykernel_30684\2029092873.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  distance_frame = distance_frame.append({
C:\Users\NicoDöring\AppData\Local\Temp\ipykernel_30684\2029092873.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  distance_frame = distance_frame.append({
C:\U

,station_code,density
0,10002,4
0,4000,6
0,4001,2
0,4002,2
0,5002,6
0,5003,7
0,5004,6
0,5005,6
0,5006,6
0,5007,7


In [23]:
def calculate_station_density(station_df, threshhold):
    distance_frame = pd.DataFrame(columns=['base_station_code', 'target_station_code', 'distance'])
    density_frame = pd.DataFrame(columns=['station_code', 'density'])


    for index in permutations(station_df.index, 2):

        base_station_code = station_df.at[index[0], 'Code']
        lon1 = station_df.iloc[[index[0]]]['longitude']
        lat1 = station_df.iloc[[index[0]]]['latitude']

        target_station_code = station_df.at[index[1], 'Code']
        lon2 = station_df.iloc[[index[1]]]['longitude']
        lat2 = station_df.iloc[[index[1]]]['latitude']

        # calculate distance
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * arcsin(sqrt(a)) 
        km = 6367 * c

        distance_frame = distance_frame.append({
            'base_station_code': base_station_code,
            'target_station_code':target_station_code,
            'distance': km
        }, ignore_index=True)

        distance_frame[['base_station_code', 'target_station_code']] = distance_frame[['base_station_code', 'target_station_code']].astype('int')

    unique_base_station_codes = list(distance_frame.base_station_code.unique())

    for base_station_code in unique_base_station_codes:

        dict = {
            'station_code': base_station_code,
            'density': len(distance_frame[(distance_frame['base_station_code']==base_station_code) & (distance_frame['distance'] < threshhold)])
        }

        df2 = pd.DataFrame([dict])

        density_frame = pd.concat([density_frame, df2])

    density_frame